<a href="https://colab.research.google.com/github/SolidStill/DFx/blob/main/XenoCantoAPI_to_CSV%26HTML_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import requests
import numpy as np
import pandas as pd
import folium

""" below fn get_recent_for_country() queries the API for all recordings
from specified country - this is excessive and limited by API: comprehensive solution
would ensure all relevant entries from API's database are extracted.
See https://xeno-canto.org/explore/api API documentation for further details on
how to handle this.
"""

def get_recent_for_country(chosen_country, n):

    # API query string to return bird song recordings for a specified country
    url = f'https://xeno-canto.org/api/2/recordings?query=cnt:{chosen_country}'

    # send an HTTP GET request to the URL stored in the url variable
    response = requests.get(url)

    # parses JSON into a dict object using .json() method from 'requests' lib
    country_dict = response.json()

    # selects the relevant part of the response (where the actual juicy data is)
    country_recordings_list = country_dict['recordings'] # this a list of dicts

    # constructs dataframe using all columns in API response
    recordings_df = pd.DataFrame(country_recordings_list)

    # selects columns of particular interest and in the desired order
    selected_columns = ['en','url','file','sono','date','loc','lat','lng','id']
    selected_columns_df = recordings_df[selected_columns]


    # sorts by date - "ascending=False" for newest at top & oldest at bottom
    date_sorted_df = selected_columns_df.sort_values(by='date', ascending=False)

    # selects only the 'n' newest records,
    user_selected_dates_df = date_sorted_df.iloc[:int(n)]

    # changing column names to something more readable and unambiguous
    new_column_names = {
    'en': 'species_name',
    'url': 'recording_webpage_url',
    'file': 'audio_file_download_link',
    'sono': 'full_sonogram', # later in this function, all but the 'full' dict value from the original response's 4 dict entries under 'sono' will be removed
    'date': 'recording_date',
    'loc': 'general_location',
    'lat': 'latitude',
    'lng': 'longitude',
    'id': 'xeno_canto_recording_id'
    }

    column_name_change_df = user_selected_dates_df.rename(columns=new_column_names)

    # each 'full_sonogram' entry is a dict of 4 sizes of sonogram, this changes the entry to the URL of the largest "full" sonogram only
    column_name_change_df['full_sonogram'] = column_name_change_df['full_sonogram'].apply(lambda x: x.get("full"))

    # ensures the 'recording_webpage_url' and 'full_sonogram' column entries are prepended by 'https:'
    # (Currently, all webpage URLs I can see are starting with '//')
    columns_to_modify = ['recording_webpage_url', 'full_sonogram']

    for column in columns_to_modify:
        column_name_change_df[column] = column_name_change_df[column].astype(str).apply(
            lambda x: 'https:' + x if not x.startswith('https:') else x
        )

    # Convert latitude and longitude to numeric, but fill errors with NaN
    # (this means if conversion fails, the entry will be set to NaN and we can drop it later)
    for column in ['latitude', 'longitude']:
        column_name_change_df[column] = pd.to_numeric(column_name_change_df[column], errors='coerce')

    final_user_df = column_name_change_df

    return final_user_df

from jinja2 import Template

def plot_recordings_map(selected_recordings_df, map_filename_prefix):
    """Generates a Folium map with clickable audio and separate sonogram modals."""

    selected_recordings_df = selected_recordings_df.dropna(subset=['latitude', 'longitude'])

    mean_lat = selected_recordings_df['latitude'].mean()
    mean_lon = selected_recordings_df['longitude'].mean()

    bird_map = folium.Map(location=[mean_lat, mean_lon], zoom_start=5)

    # Create a common modal template outside the loop
    sonogram_modal_template = Template("""
        <div id="sonogramModal{{ index }}" class="modal">
            <div class="modal-content">
                <span class="close-button" onclick="closeModal('sonogramModal{{ index }}')">&times;</span>
                <img class="sonogram-image" src="{{ full_sonogram }}">
            </div>
        </div>
    """)

    # Add markers for each recording
    for index, row in selected_recordings_df.iterrows():
        audio_html = f"""
            <audio controls>
                <source src="{row['audio_file_download_link']}" type="audio/mpeg">
                Your browser does not support the audio element.
            </audio>
        """

        # Button to trigger the modal
        button_html = f"""
            <button onclick="openModal('sonogramModal{index}')">View Sonogram</button>
        """

        popup_html = f"""
            <div class="popup-content">
                <h3>{row['species_name']}</h3>
                <p>Date: {row['recording_date']}</p>
                <p>Location: {row['general_location']}</p>
                {audio_html}
                {button_html}
            </div>
        """

        # Add the modal HTML to the map's root
        bird_map.get_root().html.add_child(folium.Element(
            sonogram_modal_template.render(index=index, full_sonogram=row['full_sonogram'])
        ))

        popup = folium.Popup(html=popup_html, max_width=300)

        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=popup,
            icon=folium.Icon(color="green"),
        ).add_to(bird_map)

    # Add the JavaScript to the map header
    js = """
    <script>
        function openModal(modalId) {
            var modal = document.getElementById(modalId);
            if (modal) {
                modal.style.display = "flex";
            } else {
                console.error("Modal not found:", modalId);
            }
        }

        function closeModal(modalId) {
            var modal = document.getElementById(modalId);
            if (modal) {
                modal.style.display = "none";
            } else {
                console.error("Modal not found:", modalId);
            }
        }

        // Close modal when clicking outside of it
        window.onclick = function(event) {
            if (event.target.className === "modal") {
                event.target.style.display = "none";
            }
        }
    </script>
    """
    bird_map.get_root().html.add_child(folium.Element(js))

    # Add CSS styles to the map for the modal
    css = """
    <style>
        /* Basic popup styling */
        .popup-content {
            overflow-y: auto;
        }

        /* Modal styles */
        .modal {
            display: none;
            position: fixed;
            z-index: 1000;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            overflow: auto;
            background-color: rgba(0,0,0,0.9);
            justify-content: center;
            align-items: center;
        }

        .modal-content {
            position: relative;
            background-color: transparent;
            margin: auto;
            padding: 0;
            width: 90%;
            max-width: 100vw;
            display: flex;
            justify-content: center;
            align-items: center;
        }

        .sonogram-image {
            max-width: 100%;
            max-height: 90vh;
            object-fit: contain;
        }

        .close-button {
            color: #f1f1f1;
            position: absolute;
            top: 10px;
            right: 25px;
            font-size: 35px;
            font-weight: bold;
            cursor: pointer;
            z-index: 1001;
        }

        .close-button:hover,
        .close-button:focus {
            color: #bbb;
            text-decoration: none;
            cursor: pointer;
        }
    </style>
    """
    bird_map.get_root().header.add_child(folium.Element(css))

    # Save the map to an HTML file
    bird_map.save(map_filename_prefix)
    print(f"Map saved as {map_filename_prefix}")











""" RE below country input: need to improve UX for this string input; beyond just using .lower() and similar methods.
i.e. ideally something like elastic search interface w/ dropdown selectability
"""
user_chosen_country = input('Which country (e.g. italy) would you like to find recordings from?: '.lower())

# RE below number of records input: specific date range would be a strong improvement
user_number_of_most_recent_recordings = input('How many of the most recent recordings would you like to view?: ')

# generates the final specified dataframe and assigns to 'user_birdsong_df'
user_birdsong_df = get_recent_for_country(user_chosen_country, user_number_of_most_recent_recordings)

# resets the index so the rows are numbered according to their date of recording
user_birdsong_df.reset_index(drop=True, inplace=True)

# display the dataframe
print(user_birdsong_df)

# user input for filename prefixes - user's API parameters appended to name
prefix = (
    input("Choose a name prefix for the CSV and map files: ")
    + "_"
    + user_chosen_country
    + "_"
    + user_number_of_most_recent_recordings
)
# this outputs a CSV from the final form of user_birdsong_df
# index=False to remove the pandas index column from the output as this doesn't seem helpful to include
user_birdsong_df.to_csv(prefix+".csv",index=False)

# generates folium html with map of recording locations
plot_recordings_map(user_birdsong_df, prefix+".html")

which country (e.g. italy) would you like to find recordings from?: italy
How many of the most recent recordings would you like to view?: 42
              species_name          recording_webpage_url  \
0             Alpine Swift  https://xeno-canto.org/912141   
1            Common Cuckoo  https://xeno-canto.org/911721   
2       Common Wood Pigeon  https://xeno-canto.org/911870   
3   Eurasian Collared Dove  https://xeno-canto.org/909830   
4            Common Cuckoo  https://xeno-canto.org/910012   
5   Eurasian Collared Dove  https://xeno-canto.org/909839   
6             Pallid Swift  https://xeno-canto.org/909107   
7             Pallid Swift  https://xeno-canto.org/909106   
8             Common Swift  https://xeno-canto.org/907036   
9            Common Cuckoo  https://xeno-canto.org/910475   
10           Common Cuckoo  https://xeno-canto.org/906248   
11       Barbary Partridge  https://xeno-canto.org/901725   
12       Barbary Partridge  https://xeno-canto.org/901727   
13   